# 武将カフェ検索＆予約プラグインデモ(LangChain)
2つの異なるシステムを ChatGPT プラグインとして公開し、これを AI オーケストレーターである LangChain から呼ぶデモを構築します。「源範頼に関連するカフェ名を検索して、7/1の18時に予約に空きがあるか教えて。もし空いていたら予約しておいて。」という複数のタスク解決が必要なユーザーからの指示に回答できるようなシステムを構築します。

![](./images/04_001.png)


# 事前準備
この Python サンプルを実行するには、以下が必要です：

- Azure OpenAI Service にアクセスできる承認済み Azure サブスクリプション
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI `gpt-35-turbo` もしくは `gpt-4` モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

<div class="alert alert-block alert-success"><b>Note:</b> 高度な Reasoning を高精度に行うには GPT-4 のモデルを推奨します。</div>

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。


## セットアップ
以下のプラグインを起動しておきます。
- cafe-review-plugin
- restaurant-reservation-plugin

それぞれのプラグインディレクトリ `plugins/` に入って、以下を実行します。

```bash
pip install -r requirements.txt
```

ライブラリのインストールができたら、

```bash
python main.py
```

でサーバーを起動します。

## パッケージのインストール

In [ ]:
!pip install openai[datalib]==1.3.9
!pip install langchain==0.0.350

In [ ]:
import openai
print("openai", openai.__version__)
import langchain
print("langchain", langchain.__version__)

## 必要なライブラリと環境変数のインポート

In [ ]:
# from langchain.chat_models import ChatOpenAI #If you use OpenAI's API
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools import AIPluginTool
from langchain.agents.mrkl import prompt

from requests.exceptions import ConnectionError


## Azure OpenAI の設定

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "chat"  # 自動構築時のデフォルト設定}

In [ ]:
# If you use OpenAI's API
#import os
#os.environ["OPENAI_API_KEY"] = "<Your OpenAI KEY>"

# Tool のロード
ChatGPT Plugin をロードするには、AIPluginTool に登録します。

In [ ]:
# If you use OpenAI's API
# llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)

llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    api_version="2023-07-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_API_KEY,
    # azure_ad_token_provider=self.openai_ad_token,
    temperature=0.0,
)
tools = load_tools(["requests_all"])
plugin_urls = [
    "http://localhost:5005/.well-known/ai-plugin.json",
    "http://localhost:5006/.well-known/ai-plugin.json",
]

tools += [AIPluginTool.from_plugin_url(url) for url in plugin_urls]

# Agent の初期化

Agent には `ZERO_SHOT_REACT_DESCRIPTION` を利用します。制約事項については SUFFIX を追加しています。


In [ ]:
SUFFIX = """
'Answer should be in Japanese. Use http instead of https for endpoint.
If there is no year in the reservation, use the year 2023. 
"""

# Responsible AI MetaPrompt
# **IMPORTANT**
# If a restaurant reservation is available, must check with the user before making a reservation if yes.'

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=dict(suffix=SUFFIX + prompt.SUFFIX),
    handle_parsing_errors=True,
    max_iterations=5, # 最大試行回数
    early_stopping_method="generate",
)

# 実行
いつまで経っても答えに行きつかない場合、GPT-4 モデルを利用すると成功する確率が上がります。GPT-35-Turbo の場合は、システムメッセージを工夫すると改善される可能性があります。

In [ ]:
try:
    result = agent_chain.run("源範頼に関連するカフェ名を検索して")
    print(result)
except ConnectionError as e:
    print("すみません、わかりません。(ConnectionError)", e)
except Exception as e:
    print("すみません、わかりません。(Error)", e)

In [ ]:
try:
    result = agent_chain.run("源範頼に関連するカフェ名を検索して、7/1の18時に予約に空きがあるか教えて。もし空いていたら予約しておいて。")
    print(result)
except ConnectionError as e:
    print("すみません、わかりません。(ConnectionError)", e)
except Exception as e:
    print("すみません、わかりません。(Error)", e)


In [ ]:
try:
    result = agent_chain.run("カフェかば殿に7/1の18時に予約を取って。")
    print(result)
except ConnectionError as e:
    print("すみません、わかりません。(ConnectionError)", e)
except Exception as e:
    print("すみません、わかりません。(Error)", e)
